# Index
* [Import](#index)
* [Tokenization](#Tokenization)
* [CountVectorizer](#CountVectorizer)
* [Tfidf](#Tfidf)
* [OvsRest](#OvsRest)
* [Word2Vec](#Word2Vec)
* [BERT](#BERT)
* [USE](#USE)

In [1]:
import time
import warnings
import numpy as np
import pandas as pd
import gensim

# from gensim import (
#     corpora,
# )  # https://pypi.org/project/gensim/ #https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py
from sklearn import (
    metrics,
    feature_extraction,
    linear_model,
    model_selection,
    preprocessing,
    cluster,
    multiclass,
    svm,
    pipeline,
    exceptions
)

# import nlpk

test https://scikit-learn.org/stable/modules/multiclass.html autre stategie que OneVsRest
https://keras.io/examples/nlp/text_extraction_with_bert/

In [2]:
pip install -q jupyter-black jupyter

Note: you may need to restart the kernel to use updated packages.


In [3]:
# for black formating
import jupyter_black  # pip install jupyter-black jupyter

jupyter_black.load()
# ??jupyter_black.load #show config

In [4]:
df = pd.read_csv(
    "/kaggle/input/p5-nlp/stackoverflowCleaned.csv", index_col=0, low_memory=False
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118442 entries, 0 to 118441
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           118442 non-null  int64 
 1   Title        118442 non-null  object
 2   Body         118442 non-null  object
 3   0            118440 non-null  object
 4   1            118383 non-null  object
 5   2            118292 non-null  object
 6   3            118292 non-null  object
 7   4            76389 non-null   object
 8   5            4 non-null       object
 9   Title_clean  118389 non-null  object
 10  Code         88202 non-null   object
 11  Body_clean   118392 non-null  object
dtypes: int64(1), object(11)
memory usage: 11.7+ MB


# Tokenization

In [5]:
from ipywidgets import interact


@interact(i=(0, df.shape[0] - 1))
def test_tr_print(i):
    print(
        f'Title:{df["Title"][i]}\n{"*"*30}\nTitle_clean:{df["Title_clean"][i]}\n{"*"*30}\nBody:{df["Body"][i]}\n{"*"*30}\nBody_clean:{df["Body_clean"][i]}\n{"*"*30}\nCode:{df["Code"][i]}\n{"*"*30}\nTags:{df.loc[i,"0"]} {df.loc[i,"1"]} {df.loc[i,"2"]} {df.loc[i,"3"]} {df.loc[i,"4"]}'
    )

interactive(children=(IntSlider(value=59220, description='i', max=118441), Output()), _dom_classes=('widget-in…

In [6]:
sum(df["Title_clean"].isna())

53

In [7]:
df[df["Title_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "0", "1", "2", "3", "4"]
].head(5)

,Title,Title_clean,Body,Body_clean,0,1,2,3,4
302,Is < faster than <=?,NaN,<p>Is <code>if (a &lt; 901)</code> faster than...,example performance change loop code suppose g...,c++,c,performance,assembly,relational-operators
393,What is setup.py?,NaN,<p>What is <code>setup.py</code> and how can i...,configure,python,pypi,setup.py,python-packaging,NaN
1437,What is related_name used for?,NaN,<p>What is the <code>related_name</code> argum...,argument manytomanyfield foreignkey field exam...,python,django,foreign-keys,many-to-many,NaN
4062,"What is ""android:allowBackup""?",NaN,"<p>Since the <a href=""http://tools.android.com...",adt preview version version lint warning tell ...,android,adt,compiler-warnings,android-lint,android-backup-service
4870,Why is 0 < -0x80000000?,NaN,<p>I have below a simple program:</p>\n\n<pre>...,program condition if(bal int32_min work change...,c,signed,numeric-limits,numeric-conversion,NaN


In [8]:
print(sum(df["Body_clean"].isna()))
df[df["Body_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "Code"]
].head(5)

50


,Title,Title_clean,Body,Body_clean,Code
5,What and where are the stack and heap?,stack heap,<ul>\n<li>What are the stack and heap?</li>\n<...,NaN,NaN
168,What is The Rule of Three?,rule,<ul>\n<li>What does <em>copying an object</em>...,NaN,NaN
828,Why are these constructs using pre and post-in...,construct pre behavior,<pre><code>#include &lt;stdio.h&gt;\n\nint mai...,NaN,#include <stdio.h>\n\nint main(void)\n{\n in...
1207,How to use ADB Shell when Multiple Devices are...,use adb shell multiple device connect fail err...,<pre><code>$ adb --help\n</code></pre>\n<hr />...,NaN,$ adb --help\n -s SERIAL use device with give...
1640,What are the advantages of list initialization...,advantage list initialization brace,<pre><code>MyClass a1 {a}; // clearer and ...,NaN,MyClass a1 {a}; // clearer and less error-...


In [9]:
len(df[df["Body_clean"].isna() & df["Title_clean"].isna()])

0

In [10]:
sum((df["Title_clean"] + df["Body_clean"]).isna())

103

In [11]:
def tag_is_in(df: pd.DataFrame(), tag: str, nb_cols: int = 6):
    return eval("|".join(f'(df["{i}"] == "{tag}")' for i in range(0, nb_cols)))


def tags_are_in(df: pd.DataFrame(), tags: list):
    return pd.DataFrame({tag: tag_is_in(df, tag) for tag in tags})


# tags_are_in(df, ["javascript","java"])
tags_are_in(
    df[10001:15050], ["javascript", "java", "c#", "python", "html", "git", "php"]
)

,javascript,java,c#,python,html,git,php
10001,True,False,False,False,True,False,False
10002,False,True,False,False,False,False,False
10003,False,False,True,False,False,False,False
10004,False,False,False,False,True,False,False
10005,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
15045,False,False,False,False,False,False,False
15046,False,False,False,False,False,False,False
15047,True,False,False,False,False,False,False
15048,False,False,False,False,False,False,False


In [12]:
tags = df[["0", "1", "2", "3", "4", "5"]].stack()
target_names = [
    tag
    for tag, freq in tags.reset_index(drop=True).value_counts(normalize=True).items()
    if freq > 0.002
]
target_names

['java',
 'c#',
 'javascript',
 'python',
 'android',
 'c++',
 'ios',
 'html',
 'php',
 '.net',
 'jquery',
 'css',
 'objective-c',
 'c',
 'sql',
 'iphone',
 'asp.net',
 'mysql',
 'linux',
 'node.js',
 'arrays',
 'swift',
 'sql-server',
 'json',
 'string',
 'windows',
 'performance',
 'ruby-on-rails',
 'xcode',
 'spring',
 'ajax',
 'database',
 'asp.net-mvc',
 'algorithm',
 'ruby',
 'macos',
 'multithreading',
 'visual-studio',
 'django',
 'reactjs',
 'wpf',
 'xml',
 'bash',
 'eclipse',
 'python-3.x',
 'image',
 'unit-testing',
 'angular',
 'c++11',
 'angularjs',
 'git']

# CountVectorizer

In [13]:
# target_names = ["javascript", "java", "c#", "python", "html", "git", "php"]
df["Token"] = df["Title_clean"].fillna("") + df["Body_clean"].fillna("")
print(
    f"{len(df[[len(token.split()) > 1 for token in df['Token']]])/len(df):.5%} des questions ont plus d'un tokens"
    if len(df[df["Token"] == ""]) == 0
    else f"{len(df[df['Token'] == ''])} questions n'ont pas de token"
)
LONG = False

for min_df in [0.01, 0.05, 0.005, 0.0005, 0.00001] if LONG else [0.00001]:
    for max_df in [0.95, 0.995, 0.9995] if LONG else [0.995]:
        print(f"\n{'*'*30}\n pour min_df:{min_df}; max_df:{max_df}")
        count_vectorizer = feature_extraction.text.CountVectorizer(
            min_df=min_df, max_df=max_df, max_features=None
        )
        #         pipe = pipeline.make_pipeline(feature_extraction.text.CountVectorizer(min_df=min_df, max_df=max_df, max_features=None), feature_extraction.text.TfidfTransformer( norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False))
        #         feat_train = pipe.transform(X_train["Token"])
        #         print(feat_train.shape)

        token_vectors = count_vectorizer.fit_transform(df["Token"])
        print(
            f"\ttokens unique:\n\t\t{token_vectors.shape[1]} dans title et body regroupé"
        )

99.99409% des questions ont plus d'un tokens

******************************
 pour min_df:1e-05; max_df:0.995
	tokens unique:
		53999 dans title et body regroupé


In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    token_vectors, tags_are_in(df, target_names), random_state=42
)

# Tfidf

In [15]:
tfidf_transformer = feature_extraction.text.TfidfTransformer(
    norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
).fit(X_train)
X_train = tfidf_transformer.transform(X_train)
print(X_train.shape[0])
X_test = tfidf_transformer.transform(X_test)
print(X_test.shape[0])

88831
29611


In [16]:
# [text for text in [token for token in X_test[0:5]]]

# OvsRest

In [17]:
X_train

<88831x53999 sparse matrix of type '<class 'numpy.float64'>'
	with 2625878 stored elements in Compressed Sparse Row format>

In [18]:
y_train[0:5]

,java,c#,javascript,python,android,c++,ios,html,php,.net,...,xml,bash,eclipse,python-3.x,image,unit-testing,angular,c++11,angularjs,git
42834,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25716,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93960,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106821,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
84135,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
print("Début entrainement OvsRest")

Début entrainement OvsRest


In [20]:
%%time
warnings.filterwarnings("ignore",category=exceptions.ConvergenceWarning)
OvsRestAll = multiclass.OneVsRestClassifier(
    svm.SVC(cache_size=1080, max_iter=5_000, random_state=42)
).fit(
    X_train,
    y_train,
)

CPU times: user 2h 47min 25s, sys: 11.4 s, total: 2h 47min 36s
Wall time: 2h 47min 55s


In [21]:
print("Fin entrainement OvsRest début prédiction")

Fin entrainement OvsRest début prédiction


In [22]:
%%time
pred = OvsRestAll.predict(X_test)
pred

CPU times: user 1h 10min 16s, sys: 11.8 s, total: 1h 10min 28s
Wall time: 1h 10min 6s


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
# print(
#     metrics.classification_report(
#         y_test,
#         pred,
#         target_names=target_names,
#         zero_division=0,
#     )
# )  # zero_division for the warning

scores = pd.DataFrame(
    metrics.classification_report(
        y_test, pred, target_names=target_names, zero_division=0, output_dict=True
    )
)
scores.to_csv("/kaggle/working/result_OvsRest.csv")
scores

,java,c#,javascript,python,android,c++,ios,html,php,.net,...,image,unit-testing,angular,c++11,angularjs,git,micro avg,macro avg,weighted avg,samples avg
precision,0.760667,0.565953,0.652016,0.890451,0.927681,0.848113,0.786850,0.706767,0.859797,0.669202,...,0.574468,0.766467,0.888889,0.794393,0.966667,0.904110,0.771220,0.772529,0.763135,0.527019
recall,0.641372,0.433631,0.640942,0.703292,0.727628,0.457739,0.591848,0.428490,0.625691,0.112173,...,0.084375,0.406349,0.418301,0.283333,0.417266,0.722628,0.467300,0.404696,0.467300,0.438355
f1-score,0.695944,0.491034,0.646431,0.785882,0.815566,0.594577,0.675558,0.533523,0.724297,0.192140,...,0.147139,0.531120,0.568889,0.417690,0.582915,0.803245,0.581971,0.508451,0.557639,0.456634
support,3558.000000,3473.000000,3356.000000,2612.000000,2045.000000,1964.000000,1840.000000,1755.000000,1627.000000,1569.000000,...,320.000000,315.000000,306.000000,300.000000,278.000000,274.000000,45887.000000,45887.000000,45887.000000,45887.000000


# Word2Vec

In [24]:
df[0:50]["Token"]

0     process array process arraypiece c++ code show...
1     undo commit gitcommit file git push commit ser...
2     delete git branchdelete remote origin bugfix b...
3     difference git pull git fetchdifference git pu...
4     operator c++read hidden features dark corners ...
5                                            stack heap
6     force git pull overwrite fileforce overwrite f...
7     check element hide jquerytoggle visibility ele...
8     use javascript reasoningrun javascript code cr...
9     _ _ _ = _ _include statement try close questio...
10    modify exist commit messagewrite thing commit ...
11    javascript closure workexplain javascript clos...
12    revert git repository commitrevert state snaps...
13    java pass reference pass valuethink java use p...
14    var functionname function vs function function...
15           difference string string c #difference use
16                          metaclasse pythonmetaclasse
17    find file contain text string linuxfind fi

In [25]:
sentences = [gensim.utils.simple_preprocess(text) for text in df[0:20000]["Token"]]
#sentences

In [26]:
# gensim.corpora.Dictionary(token["Token"])

In [27]:
from tensorflow import keras

In [28]:
print("Build & train Word2Vec model ...")
w2v_min_count = 1
w2v_size = 300
w2v_window = 5
w2v_epochs = 100
maxlen = 24
w2v_model = gensim.models.Word2Vec(
    min_count=w2v_min_count, window=w2v_window, vector_size=w2v_size, seed=42, workers=1
)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 35610
Word2Vec trained


In [29]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(sentences), maxlen=maxlen, padding="post"
)

num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Number of unique words: 35611


# BERT

# USE